In [1]:
import numpy as np

Step by step, k = 1:

In [2]:
p = 5
k = 1

factor_loadings = np.array([2,2,2,2,2])

factor_vector = np.random.multivariate_normal(np.zeros(k), np.eye(k))

specific_variance = [2,3,5,8,9]
u = np.random.multivariate_normal(np.zeros(p), np.diag(specific_variance))

mu = np.array([70, 70, 50, 30, 30])

In [3]:
x = factor_loadings * factor_vector + u + mu
x

array([71.31798067, 69.13652953, 50.21527753, 27.89618382, 29.40788381])

Define function to simulate:

In [16]:
def sim_factor_model(loadings, specific_variance, mu, nsim=1, verbose=True):
    """
    Parameters
    ---
        loadings:           (p, k) matrix
        specific_variance:  (p, p) diagonal matrix of specific variances
        mu:                 (p, 1) vector of means
        nsim:               How many observations should be simulated

    Returns
    ---
        (n, p) matrix of observations from the specified factor model

    """
    k = loadings.shape[1]
    p = len(specific_variance)
    print(f"{k=} {p=}") if verbose else ""
    X = []
    for _ in range(nsim):
        factor_vector = np.random.multivariate_normal(np.zeros(k), np.eye(k))
        u = np.random.multivariate_normal(np.zeros(p), np.diag(specific_variance))

        X.append(loadings @ factor_vector + u + mu)

    return np.array(X)

General k = 1:

In [17]:
loadings = np.array([[2, 2, 2, 2, 2]]).T
specific_variance = np.array([2, 2, 10, 1, 1])
mu = np.array([10, 20, 30, 40, 50])

X = sim_factor_model(loadings, specific_variance, mu, nsim=10**4)

k=1 p=5


Testing hypothesis:
$$
S \approx \Lambda \Lambda' + \Psi
$$

In [19]:
cov = np.cov(X.T)
print("Covariance matrix (ROUNDED):\n", cov.round(2), sep="")

approximation = loadings @ loadings.T + np.diag(specific_variance)
print("Approximation:\n", approximation, sep="")

Covariance matrix (ROUNDED):
[[ 5.89  3.91  4.08  3.95  4.  ]
 [ 3.91  5.98  4.06  3.96  3.98]
 [ 4.08  4.06 14.14  4.1   4.09]
 [ 3.95  3.96  4.1   4.99  3.99]
 [ 4.    3.98  4.09  3.99  5.02]]
Approximation:
[[ 6  4  4  4  4]
 [ 4  6  4  4  4]
 [ 4  4 14  4  4]
 [ 4  4  4  5  4]
 [ 4  4  4  4  5]]


General k = 2:

In [21]:
loadings = np.array([[2, 2, 2, 2, 2],
                     [1, 1, 0, -1, -1]]).T
specific_variance = np.array([2, 2, 10, 1, 1])
mu = np.array([10, 20, 30, 40, 50])

X = sim_factor_model(loadings, specific_variance, mu, nsim=10**4)

k=2 p=5


In [22]:
cov = np.cov(X.T)
print("Covariance matrix (ROUNDED):\n", cov.round(2), sep="")

approximation = loadings @ loadings.T + np.diag(specific_variance)
print("Approximation:\n", approximation, sep="")

Covariance matrix (ROUNDED):
[[ 7.11  5.12  4.04  3.    3.  ]
 [ 5.12  7.15  4.02  3.09  3.09]
 [ 4.04  4.02 14.04  3.97  4.  ]
 [ 3.    3.09  3.97  5.95  4.98]
 [ 3.    3.09  4.    4.98  6.03]]
Approximation:
[[ 7  5  4  3  3]
 [ 5  7  4  3  3]
 [ 4  4 14  4  4]
 [ 3  3  4  6  5]
 [ 3  3  4  5  6]]


Hypothesis holds!

Lets try breaking constraint (9.2.7):

$$
\Lambda' \Psi^{-1} \Lambda \text{ must be diagonal}
$$
It implies that
$$
\left( \Lambda' \Psi^{-1} \Lambda \right)_{12} = \frac{2}{\psi_{11}} + \frac{2}{\psi_{22}} - \frac{1}{\psi_{44}} - \frac{1}{\psi_{55}} = 0
$$

In [23]:
off_diagonal = 2/1 + 2/2 - 1/3 - 1/4
off_diagonal == 0

False

In [24]:
loadings = np.array([[2, 2, 2, 2, 2],
                     [1, 1, 0, -1, -1]]).T
specific_variance = np.array([1, 2 , 90, 3, 4]) # Here the constraint 9.2.7 breaks
mu = np.array([10, 20, 30, 40, 50])

X = sim_factor_model(loadings, specific_variance, mu, nsim=10**4)

cov = np.cov(X.T)
print("Covariance matrix (ROUNDED):\n", cov.round(2), sep="")

approximation = loadings @ loadings.T + np.diag(specific_variance)
print("Approximation:\n", approximation, sep="")

k=2 p=5
Covariance matrix (ROUNDED):
[[ 6.16  5.11  4.04  3.05  3.04]
 [ 5.11  7.    4.18  3.03  3.  ]
 [ 4.04  4.18 95.44  4.03  4.45]
 [ 3.05  3.03  4.03  8.12  5.07]
 [ 3.04  3.    4.45  5.07  9.07]]
Approximation:
[[ 6  5  4  3  3]
 [ 5  7  4  3  3]
 [ 4  4 94  4  4]
 [ 3  3  4  8  5]
 [ 3  3  4  5  9]]


Note that this is just to remove the indeterminacy - The model stills works. (pretty sure at least :S)